In [9]:
import tensorflow as tf
from keras.src.models import Sequential
from keras.src.losses.losses import CategoricalCrossentropy
from keras.src.layers import Dense, TimeDistributed
from keras.api.optimizers import SGD
from keras.src.callbacks import TensorBoard
from keras.api.models import save_model, load_model
from models.functions import open_csv, save_as_csv
from models.constants import Paths, Headers, Constants
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np

ModuleNotFoundError: No module named 'models'

In [10]:
class MiniBatchSGD:
    def __init__(self, learning_rate=0.01, clip_value=0.5):
        self.learning_rate = learning_rate
        self.clip_value = clip_value
        self.optimizer = SGD(learning_rate=learning_rate)

    def train_step(self, model, features, labels, batch_size):
        ds = tf.data.Dataset.from_tensor_slices((features, labels)).batch(batch_size=batch_size)

        for batch_features, batch_labels in ds:
            with tf.GradientTape() as tape:
                predictions = model(batch_features, training=True)   # Set training True for dropout etc.
                loss = self.loss_function(batch_labels, predictions)

            # Calculate gradients with gradient clipping
            grads = tape.gradient(loss, model.trainable_variables)
            clipped_grads = [tf.clip_by_value(grad, -self.clip_value, self.clip_value) for grad in grads]

            # Apply gradients with clipping
            self.optimizer.apply_gradients(zip(clipped_grads, model.trainable_variables))

    def loss_function(self, labels, predictions):
        return CategoricalCrossentropy(labels, predictions)

In [13]:
model = Sequential()
model.add(TimeDistributed(Dense(10, activation="relu")))
model.add(Dense(1))  # Output layer for predicting volume

ds: pd.DataFrame = open_csv(Paths.filtered_csv, Headers.needed_headers)

dep = ds["Vol"]  # dependent var
ds.drop(columns="Vol", inplace=True)  # independent var

dep_max = dep.max()
dep_min = dep.min()

for c in Constants.categories:
    ds[c] = ds[c].astype("category", copy=False).cat.codes  # Adjusting ind var categories to int
mms = MinMaxScaler()
dep = mms.fit_transform(dep.values.reshape(-1, 1))  # Normalizing dep var from 0 to 1

x_train, x_test, y_train, y_test = train_test_split(ds, dep, test_size=0.2)

model.compile(optimizer="sgd", loss="mse")
model.fit(ds, dep, batch_size=32, epochs=50)

FileNotFoundError: [Errno 2] No such file or directory: '.../data/filtered.csv'

In [12]:
x_predict = model.predict(x_test)
output = model.evaluate(x_predict, y_test)

NameError: name 'x_test' is not defined